## Adding data from FOIA requests

The data in the Chicago Data Portal only includes employee reimbursements paid after Jan. 1 of the previous calendar year. I am in the process of submitting FOIA requests to get data from further back, so that my site can host both historical and current data in one place.

After importing necessary packages, I define the path to the data and join path with anything files ending in the extension .xlsx (each file shared with my had the reimbursements paid in one quarter of one requested year, so my first return of 5 years of transactions gave me 20 excel files).

In [19]:
import pandas as pd
import glob
import os

In [31]:
path = '/Users/hgorledeenn/Desktop/Chicago-Reimbursements-site/foia_data'
all_files = glob.glob(os.path.join(path, "*.xlsx"))
columns_to_use = ['Dept Code', 'Pymt Date', 'Vendor Name', 'Vendor Number', 'Voucher (Batch) Number', 'Invoice Line Nbr', 'Invoice Line Amount', 'Invoice Number', 'Invoice Date', 'Invoice Description']

## Reading in the Data

I use a for loop to read in each excel file and merge them into one large file.

In [32]:
df_list = []

for filename in all_files:
    df_small = pd.read_excel(filename, usecols=columns_to_use)
    df_list.append(df_small)
    df = pd.concat(df_list, ignore_index=True)

df.head()

,Dept Code,Pymt Date,Vendor Name,Vendor Number,Voucher (Batch) Number,Invoice Line Nbr,Invoice Line Amount,Invoice Number,Invoice Date,Invoice Description
0,1.0,2021-02-05,"HALL, PATRICK J",10530390.0,PV01210100003,1,305.70,00003-2021,2021-01-25,Travel to Springfield 1/8-1/13/2021
1,6.0,2021-03-29,"NOLAN, CARLETON",10551496.0,PV06190600581,1,1093.40,52585266,2019-06-11,LODGING REIMBURSEMENT TO CIO CARLETON NOLAN/NY...
2,15.0,2021-01-05,"CARDONA, FELIX JR.",10560242.0,PV15201554053,1,18.00,194B0039969,2020-12-18,"Parking, gasoline, insurance and maintenance o..."
3,15.0,2021-01-05,"CARDONA, FELIX JR.",10560242.0,PV15201554068,1,55.00,029046,2020-12-18,"Postage, shipping and messenger fees"
4,15.0,2021-01-05,"CARDONA, FELIX JR.",10560242.0,PV15201554068,1,32.98,9569293,2020-12-21,Office supplies


## Data Wrangling

I did some data cleaning, like filtering to remove the 'Sum:' rows and rows where the invoice amount is $0. I also changed column types to make them most useful for me (or just easier to read).

In [33]:
df = df[df['Invoice Line Nbr'] != 'Sum:']
df = df[df['Invoice Line Amount'] != 0]

df['Dept Code'] = df['Dept Code'].astype('Int64')
df['Vendor Number'] = df['Vendor Number'].astype('object')

df.dtypes

Dept Code                          Int64
Pymt Date                 datetime64[ns]
Vendor Name                       object
Vendor Number                     object
Voucher (Batch) Number            object
Invoice Line Nbr                  object
Invoice Line Amount              float64
Invoice Number                    object
Invoice Date              datetime64[ns]
Invoice Description               object
dtype: object

Some of the rows of the dataset represent only parts of an invoice paid (represented by a value>1 for the ```Invoice Line Nbr``` column). I grouped by the ```Voucher (Batch) Number``` and ```Pymt Date``` columns to create a new dataframe of the total invoice amount, then merged in the other columns from the original dataframe.

In [34]:
df_sums = df.groupby(['Voucher (Batch) Number', 'Pymt Date', 'Invoice Description'])['Invoice Line Amount'].sum()

df = pd.merge(df_sums,df[['Voucher (Batch) Number','Dept Code', 'Pymt Date', 'Vendor Name', 'Vendor Number', 'Invoice Number', 'Invoice Date', 'Invoice Description']],on='Voucher (Batch) Number', how='left')

df = df.drop_duplicates(subset=['Voucher (Batch) Number', 'Invoice Line Amount'], keep='first')

df.head()

,Voucher (Batch) Number,Invoice Line Amount,Dept Code,Pymt Date,Vendor Name,Vendor Number,Invoice Number,Invoice Date,Invoice Description
0,PV01180100144,394.80,1,2019-02-21,"COLLINS, ADAM",10230254.0,00144,2018-11-14,Travel 11/9-11/11/18
2,PV01180100145,301.27,1,2019-02-22,"SCHWESKA, PATRICK R",10231322.0,00145,2018-12-03,Travel 11/26-11/29/18
4,PV01180100145,276.91,1,2019-02-22,"SCHWESKA, PATRICK R",10231322.0,00145,2018-12-03,Travel 11/26-11/29/18
6,PV01180100146,274.91,1,2019-02-22,"NEWBERN, TIFFANY G",10202626.0,00146,2018-12-06,Travel 3/5-3/7/18
10,PV01180100146,329.90,1,2019-02-22,"NEWBERN, TIFFANY G",10202626.0,00146,2018-12-06,Travel 3/5-3/7/18


I also went through to change the ```Dept Code``` column from the department's number to its name. I used the key at the start of the [2025 Annual Appropriation Ordinance](https://www.chicago.gov/content/dam/city/depts/obm/supp_info/2025Budget/2025_Ordinance_Book_webVersion.pdf).

In [38]:
dept_names = {1: 'OFFICE OF THE MAYOR',
              3: 'Office of the Inspector General',
              5: 'Office of Budget and Management',
              6: 'Department of Technology and Innovation',
              15: 'City Council',
              21: 'Department of Housing',
              23: 'Department of Cultural Affairs and Special Events',
              25: 'Office of City Clerk',
              27: 'Department of Finance',
              28: "City Treasurer's Office",
              30: 'Department of Administrative Hearings',
              31: 'Department of Law',
              33: 'Department of Human Resources',
              35: 'Department of Procurement Services',
              38: 'Department of Fleet and Facility Management',
              39: 'Board of Election Commissioners',
              41: 'Chicago Department of Public Health',
              45: 'Chicago Commission on Human Relations',
              48: "Mayor's Office for People with Disabilities",
              50: 'Department of Family and Support Services',
              51: 'Office of Public Safety ADministration',
              54: 'Department of Planning and Development',
              55: 'Chicago Police Board',
              57: 'Chicago Police Department',
              58: 'Office of Emergency Management and Communications',
              59: 'Chicago Fire Department',
              60: 'Civilian Office of Police Accountability',
              62: 'Community Commission for Public Safety and Accountability',
              67: 'Department of Buildings',
              70: 'Department of Business Affairs and Consumer Protection',
              72: 'Department of Environment',
              73: 'Chicago Animal Care and Control',
              77: 'License Appeal Commission',
              78: 'Board of Ethics',
              81: 'Department of Streets and Sanitation',
              84: 'Chicago Department of Transportation',
              85: 'Chicago Department of Aviation',
              88: 'DEpartment of Water Management',
              91: 'Chicago Public Library',
              99: 'Finance General'}


df_test = df['Dept Code']
# df_test['Dept Code'] = df_test['Dept Code'].map(dept_names)
# df_test.head()

df_test.value_counts()

Dept Code
57    7659
15    2252
41    1707
59    1426
84    1146
88    1104
33     338
31     327
1      265
85     249
70     234
38     129
54     129
27     114
28     106
30     102
48      89
50      85
45      70
91      66
60      65
58      49
51      47
23      44
78      40
35      37
6       26
21      25
5       22
25      18
73      17
67      12
62       8
55       6
39       3
47       1
81       1
40       1
Name: count, dtype: Int64

### **47** and **40** appear in the above list but not the 2025 ordinance table of contents
### **3** and **72** and **77** and **99** are in the 2025 ordinance table of contents but not in the above list